<a href="https://colab.research.google.com/github/axstiz/AD_practices/blob/master/qq/Homeworks/T2S/salute_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подлкючение нужных библиотек

In [24]:
!pip install -q --upgrade gigachat langchain-gigachat gradio ffmpeg-python requests python-dotenv pydub scipy pyTelegramBotAPI langchain

In [25]:
import os
import uuid
import json
import requests
import subprocess
import langchain
import scipy.io.wavfile
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Audio as IPythonAudio

langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, AIMessage

## Авторизация

In [33]:
from google.colab import userdata

auth_giga = userdata.get("SBER_AUTH")
auth_speech = userdata.get("SBER_SPEECH")

In [34]:
def get_salute_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """Получает временный токен доступа для SaluteSpeech."""
    rq_uid = str(uuid.uuid4())
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }
    payload = {'scope': scope}

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response.raise_for_status()
        return response.json()['access_token']
    except Exception as e:
        print(f"Ошибка получения токена SaluteSpeech: {e}")
        return None

salute_token = get_salute_token(auth_speech)
if salute_token:
    print("Token получен успешно.")

Token получен успешно.


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(




```
# Выбран кодовый формат
```

## Обработка аудио

In [35]:
def convert_audio(input_path, output_path, target_rate=16000):
    """
    Конвертирует аудио в формат: WAV, Mono, 16000Hz (оптимально для SaluteSpeech).
    Использует ffmpeg через subprocess.
    """
    try:
        command = [
            'ffmpeg',
            '-y',             # Перезаписать выходной файл
            '-i', input_path, # Входной файл
            '-ac', '1',       # 1 аудио канал (моно)
            '-ar', str(target_rate), # Частота дискретизации
            '-f', 'wav',      # Формат контейнера
            output_path
        ]
        # Запускаем ffmpeg, подавляя вывод в консоль
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception as e:
        print(f"Ошибка конвертации аудио: {e}")
        return False

In [36]:
def stt(file_path, token):
    """
    Speech-to-Text,
    Распознает речь из аудиофайла.
    """
    url = "https://smartspeech.sber.ru/rest/v1/speech:recognize"

    # Сначала конвертируем файл в правильный формат (16k, mono)
    converted_path = "temp_converted.wav"
    if not convert_audio(file_path, converted_path):
        return None

    # Указываем audio/x-wav, так как отправляем файл с заголовком
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "audio/x-wav"
    }

    try:
        with open(converted_path, "rb") as audio_file:
            audio_data = audio_file.read()

        response = requests.post(url, headers=headers, data=audio_data, verify=False)

        if response.status_code == 200:
            # Ответ может содержать несколько гипотез, берем первую
            result_json = response.json()
            return result_json.get("result", [""])
        else:
            print(f"STT API Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"STT Connection Error: {e}")
        return None

def tts(text, token, file2save='output.wav', voice='Bys_24000'):
    """
    Text-to-Speech,
    Синтезирует речь из текста.
    """
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/text"
    }
    params = {
        "format": "wav16",
        "voice": voice
    }

    try:
        response = requests.post(url, headers=headers, params=params, data=text.encode('utf-8'), verify=False)
        if response.status_code == 200:
            with open(file2save, 'wb') as f:
                f.write(response.content)
            return file2save
        else:
            print(f"TTS API Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"TTS Connection Error: {e}")
        return None

## Интерфейс для бота

In [37]:
class AudioChat:
    def __init__(self, auth_giga, salute_token):
        self.auth_giga = auth_giga
        self.salute_token = salute_token
        self.history = []
        self.llm = GigaChat(
            credentials=auth_giga,
            verify_ssl_certs=False,
            model="GigaChat-Pro"
        )

    def get_gpt_response(self, text_input):
        """Получает ответ от LLM и обновляет историю."""
        if not text_input:
            return "Я вас не расслышал."

        self.history.append(HumanMessage(content=text_input))
        try:
            response = self.llm.invoke(self.history)
            answer = response.content
            self.history.append(AIMessage(content=answer))
            return answer
        except Exception as e:
            return f"Ошибка GigaChat: {str(e)}"

    def process_voice_pipeline(self, audio_file_path):
        """
        Полный цикл:
        1. Аудио -> Текст (STT)
        2. Текст -> LLM -> Ответ (AI)
        3. Ответ -> Аудио (TTS)
        """
        # 1. Распознавание
        recognized_list = stt(audio_file_path, self.salute_token)
        if not recognized_list:
            return "Ошибка распознавания.", None

        user_text = recognized_list[0]
        print(f"Пользователь сказал: {user_text}")

        # 2. Генерация ответа
        ai_text = self.get_gpt_response(user_text)
        print(f"Ответ AI: {ai_text}")

        # 3. Синтез речи
        output_audio_file = "response.wav"
        tts(ai_text, self.salute_token, output_audio_file)

        return ai_text, output_audio_file

voice_bot = AudioChat(auth_giga, salute_token)

## Упражнение 1: Интерфейс Gradio
Создадим веб-интерфейс, который принимает микрофонную запись, обрабатывает её и возвращает текстовый и аудио ответ.

In [38]:
def gradio_handler(audio):
    if audio is None:
        return "Нет аудио", None

    # Gradio передает аудио как кортеж (sample_rate, data)
    sr, data = audio

    # Сохраняем входящее аудио во временный файл
    temp_wav = "gradio_input.wav"
    scipy.io.wavfile.write(temp_wav, sr, data)

    # Обрабатываем через наш класс
    # (Функция process_voice_pipeline внутри себя сделает конвертацию в 16k mono)
    text_response, audio_response_path = voice_bot.process_voice_pipeline(temp_wav)

    return text_response, audio_response_path

# Описание интерфейса
iface = gr.Interface(
    fn=gradio_handler,
    inputs=gr.Audio(sources=["microphone"], type="numpy", label="Нажмите Record и говорите"),
    outputs=[
        gr.Textbox(label="Текст ответа"),
        gr.Audio(label="Голосовой ответ", autoplay=True)
    ],
    title="GigaChat Voice Assistant",
    description="Скажите что-нибудь, и модель ответит вам голосом!"
)

# Запуск
iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9ba6c94176fcdf52b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Пользователь сказал: Привет, дорогой GigaChat, если у тебя закончились токены для ответа мне иди нахуй.


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Ответ AI: Генеративные языковые модели не обладают собственным мнением — их ответы являются обобщением информации, находящейся в открытом доступе. Чтобы избежать ошибок и неправильного толкования, разговоры на чувствительные темы могут быть ограничены.


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Пользователь сказал: Привет, как дела?
Ответ AI: Всё отлично! Готов обсудить любые интересные тебе темы.


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9ba6c94176fcdf52b0.gradio.live


## Упражнение 2: Tg-бот
Создадим бота в Telegram, которому можно отправлять голосовые сообщения.

In [39]:
import telebot
from pydub import AudioSegment

TG_TOKEN = userdata.get("TG_TOKEN")

bot = telebot.TeleBot(TG_TOKEN)

@bot.message_handler(commands=['start'])
def welcome(message):
    bot.send_message(message.chat.id, "Привет-привет! Отправь мне голосовое сообщение и я отвечу тебе!")

@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    try:
        # 1. Скачивание файла
        file_info = bot.get_file(message.voice.file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Telegram сохраняет в .ogg (opus)
        ogg_filename = f"user_{message.chat.id}.ogg"
        with open(ogg_filename, 'wb') as new_file:
            new_file.write(downloaded_file)

        # 2. Конвертация в WAV для SaluteSpeech (используем pydub или ffmpeg)
        wav_filename = f"user_{message.chat.id}.wav"
        # Pydub удобен для конвертации форматов
        sound = AudioSegment.from_ogg(ogg_filename)
        # Принудительно ставим 16000Гц и моно
        sound = sound.set_frame_rate(16000).set_channels(1)
        sound.export(wav_filename, format="wav")

        bot.send_message(message.chat.id, "⏳ Думаю...")

        # 3. Обработка через наш класс
        # (stt вызовет convert_audio, но мы уже сконвертировали, это не страшно)
        ai_text, audio_response_path = voice_bot.process_voice_pipeline(wav_filename)

        # 4. Отправка ответа
        if ai_text:
            bot.send_message(message.chat.id, f"🤖: {ai_text}")

        if audio_response_path and os.path.exists(audio_response_path):
            with open(audio_response_path, 'rb') as voice:
                bot.send_voice(message.chat.id, voice)

        if os.path.exists(ogg_filename): os.remove(ogg_filename)
        if os.path.exists(wav_filename): os.remove(wav_filename)

    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка: {e}")
        print(e)

print("Telegram бот запущен...")
bot.polling(non_stop=True)

Telegram бот запущен...


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Пользователь сказал: Привет, как дела?
Ответ AI: Привет! У меня всё хорошо, полон энергии и готов пообщаться. А как твои дела?


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Пользователь сказал: У меня тоже все хорошо, потому что ты работаешь. Рад, что ты ответил.
Ответ AI: Это взаимно — рад, что моё присутствие делает твой день лучше! Если захочешь поговорить ещё или задать вопрос — обращайся в любое время.


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Пользователь сказал: Хорошо, спасибо. Давай, пока, пока.
Ответ AI: Пока! Обращайся снова, когда захочешь. Хорошего дня!


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
